# Theoretical calculations for state dependent Bragg specostropy on 6Li

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.constants import h, hbar, c, epsilon_0
import collections
from sympy.physics.quantum.cg import (Wigner3j, Wigner6j)

## Zeeman splitting

In [ ]:
def dv(B, m_j, g_j, mu_B):
    return mu_B*g_j*m_j*B

In [ ]:
g_J = np.array([2.0023010, 0.6668, 1.335]) # 2S, 2P1/2, 2P3/2
mu_B_hbar = 1.4 #MHz/G
B = np.arange(0, 1000, 50) #G

In [ ]:
fig, ax = plt.subplots()
ax.plot(B, dv(B, 1/2, g_J[1], mu_B_hbar), label='m_J = 1/2')
ax.plot(B, dv(B, -1/2, g_J[1], mu_B_hbar), label='m_J = -1/2')
plt.xlabel('B [G]')
plt.ylabel('Δv [MHz]')
plt.legend()
#plt.savefig('.\\plots\\Zeeman_2P_1-2')
plt.show()

In [ ]:
fig, ax = plt.subplots()
ax.plot(B, dv(B, 3/2, g_J[2], mu_B_hbar), label='m_J = 3/2')
ax.plot(B, dv(B, 1/2, g_J[2], mu_B_hbar), label='m_J = 1/2')
ax.plot(B, dv(B, -1/2, g_J[2], mu_B_hbar), label='m_J = -1/2')
ax.plot(B, dv(B, -3/2, g_J[2], mu_B_hbar), label='m_J = -3/2')
plt.xlabel('B [G]')
plt.ylabel('Δv [MHz]')
plt.legend()
#plt.savefig('.\\plots\\Zeeman_2P_3-2')
plt.show()

## Hyperfine splitting of the Groundstate

In [ ]:
# Breit-Rabi formula
def BR(B, F, m_F): # B in Gaus, return in MHz
    a = (g_e-g_I)*mu_B*B*1e-4/(h*vHFS)
    root = 0
    if(F!=abs(m_F) or F<I+1/2):
        root = np.sqrt(1 + 4*m_F/(2*I+1)*a + a**2)
    else:
        root = 1 + m_F/abs(m_F)*a
    return (-vHFS/(2*(2*I+1)) + g_I*m_F*mu_B*B*1e-4/h + vHFS*(F-1)*root)*1e-6

def state_calc(F, m_F): # returns an int from 1 to 6, where 1 is the state with the lowest Energy
    if F==1/2:
        return int(3/2-m_F)
    if F==3/2:
        return int(9/2+m_F)

def get_F(state):
    if state <= 2:
        return 1/2
    if state > 2:
        return 3/2

def get_m_F(state):
    if state <= 2:
        return -state+3/2
    if state > 2:
        return state-9/2

def state_spacing(state1, state2, B):
    ans = BR(B, get_F(state1), get_m_F(state1)) - BR(B, get_F(state2), get_m_F(state2))
    return 'The Frequency between |{}> and |{}> at B = {} Gauss is {} MHz'.format(state1, state2, B, ans)

In [ ]:
# Constants
vHFS = 228205260 # HFS for B=0 in Hz
I = 1 # Corespin
g_I = -0.0004476540 # Lande-g Factor core
g_e = 2.0023193043737 # Lande-g Factor electrons
mu_B = 927.400949e-26  # Bohr magneton

In [ ]:
B = np.linspace(0, 1000, 1000) # in G
HFS = collections.namedtuple('HFS', 'v F m_F')
data = []
for i in range(1,7):
    F = get_F(i)
    m_F = get_m_F(i)
    data.append(HFS(v=BR(B, F, m_F), F=F, m_F=m_F))

fig, ax = plt.subplots(figsize=(16, 9))
for n in reversed(range(6)):
    ax.plot(B, data[n].v, label='|{}>'.format(n+1))
ax.set_xlabel('Magnetic field strength [G]')
ax.set_ylabel('Δv [MHz]')
ax.set_title('Breit-Rabi diagram for Li-6')
ax.legend()
#plt.savefig('.\\plots\\Zeeman_2S')
plt.show()

In [ ]:
print(state_spacing(1, 2, 1000))
print(state_spacing(2, 3, 1000))
print(state_spacing(1, 2, 832))
print(state_spacing(2, 3, 832))

## Polarizabilities

In [ ]:
# Constants
Gamma = 36.898e6 # s^-1
D1_wavel = 670.992421e-9 # m
D2_wavel = 670.977338e-9 # m
D1 = 446.789634e12 # Hz
D2 = 446.799677e12 # Hz
Delta = D1-D2
#-np.pi*epsilon_0*c**3*Gamma/((2*np.pi*D2)**3*1.64877727436e-41)

In [ ]:
def alpha(v, p, F, m_F, g_F): # in a.u., magnetic field is not taken into account
    return -np.pi*epsilon_0*c**3*Gamma/(2*np.pi*D2)**3 * ((2+p*g_F*m_F)/v + (1-p*g_F*m_F)/(v-Delta)) / 1.64877727436e-41

def g_F_calc(F):
    J = F - I
    return g_J[0] * (F * (F + 1) - I * (I + 1) + J * (J + 1)) / (2 * F * (F + 1))

def plot(v, p):
    fig, ax = plt.subplots(2, figsize=(10, 10))
    for i in [1, 2]: # States 1 & 2
        m_F = get_m_F(i)
        ax[0].plot(v*1e-9, alpha(v, p=p, F=1/2, m_F=m_F, g_F=g_F_calc(F=1/2)), label='|{}>'.format(i))
        
    for n in [1, 3]: # States 1 & 3
        F = get_F(n)
        m_F = get_m_F(n)
        ax[1].plot(v*1e-9, alpha(v, p=p, F=F, m_F=m_F, g_F=g_F_calc(F=F)), label='|{}>'.format(n))
    
    for k in range(2):
        ax[k].set_ylim(5e8, -5e8)
        ax[k].set_ylabel('α [a.u.]')
        ax[k].legend()
        #ax[k].axvline(Delta*1e-9, ls='--', color='gray') # D1
        #ax[k].axvline(0, linestyle='--', color='gray') # D2
    x_ticks = np.array([Delta, Delta/2, 0])
    x_labels = np.around(freq_to_wavel(x_ticks), decimals=3)
    ax[0].set_xticks(x_ticks*1e-9, x_labels)
    ax[0].set_title('Polarization {}'.format(format_pol(p)))
    ax[0].set_xlabel('λ [nm]')
    ax[1].set_xlabel('Δv [GHz]')
    plt.show()

def freq_to_wavel(freq): # frequency in offset from D2 in Hz, wavelength in nm
    return c/(freq+D2)*1e9

def format_pol(p):
    for pol, string in enumerate(['σ-', 'π', 'σ+'], start=-1):
        if p == pol:
            return string

In [ ]:
v = np.linspace(-13e9, 3e9, 100000) # Hz
# maybe v/2pi
plot(v, p=0)

In [ ]:
plot(v, p=-1)

In [ ]:
plot(v, p=1)

## Scattering rates

In [ ]:
# Constants
s = 0.01 # I/I_s Laser Intensity / saturation Intensity
rabi_freq = Gamma*np.sqrt(1/2*s)

In [ ]:
def rabi_fre(v, F1, m_F1, F2, m_F2, p):
    I = 1
    J1 = F1 - I
    J2 = F2 - I
    L = 0
    S1 = J1
    S2 = J2
    w3j = Wigner3j(F1, 1, F2, -m_F1, p, m_F2)
    w6j_1 = Wigner6j(J1, F1, I, F2, J2, 1)
    w6j_2 = Wigner6j(L, J1, S1, J2, L, 1)
    exp = 1/2*p*(1+p)+1+2*F1-m_F1+I+J2+S1+L+J1
    root = np.sqrt(3*freq_to_wavel(v)**3*Gamma*I/(4*np.pi**2*c*hbar))
    root_num = np.sqrt((2*F1+1)*(2*F2+1)*(2*J1+1)*(2*J2+1)*(2*L+1))
    return (-1)**exp*root*root_num*w3j*w6j_1*w6j_2

def scattering_rate(v):
    ans = 0
    for w in [D1, D2]:
        ans += rabi_freq**2/Gamma/(1+2*rabi_freq**2/Gamma**2+4*(v-w)**2/Gamma**2)
    return ans

In [ ]:
fig, ax = plt.subplots(figsize=(16, 9))
plt.plot(v, scattering_rate(v))
plt.show